## **AlexNet网络 - 深度卷积神经网络**


**参考文献：** [1] Krizhevsky, A., Sutskever, I., & Hinton, G. E. (2012). Imagenet classification with deep convolutional neural networks. In Advances in neural information processing systems (pp. 1097-1105).


<img src="AlexNet.png"></img>

神经网络可以直接基于图像的原始像素进行分类。这种称为端到端（end-to-end)的方法节省了很多中间步骤。然而，在很长一段时间里更流行的是研究者通过勤劳与智慧所设计并生成的手工特征。
这类图像分类研究的主要流程是：1）获取图像数据集；2）使用已有的特征提取函数生成图像的特征；3）使用机器学习模型对图像的特征分类。
当时认为的机器学习部分仅限最后这一步。


2012 AlexNet 使用了8层卷积神经网络，它首次证明了学习到的特征可以超越手工设计的特征，从而一举打破计算机视觉研究的现状。

AlexNet 与 LeNet 的设计理论非常相似，但是也有显著的区别。

**第一，与相对较小的LeNet相比，AlexNet包含8层变换，其中有5层卷积和2层全连接隐藏层，以及1个全连接输出层。**

 Alex 第一层中的卷积窗口形状是 $11\times 11$。 因为ImageNet中绝大多数图像的高和宽均比MNIST图像的高和宽大10倍以上，ImageNet图像的物体占用更多的像素，所以需要更大的卷积窗口来捕获物体。
 
 第二层中的卷积窗口形状减小到 $5\times 5$，之后全采用 $3\times 3$。此外，第一、第二和第五个卷积层之后都使用了窗口形状为$3\times 3$、步幅为2的最大池化层。而且，AlexNet使用的卷积通道数也大于LeNet中的卷积通道数数十倍。
 
 紧接着，最后一个卷积层的是两个输出个数为4096的全连接层。这两个巨大的全连接层带来将近1GB 的模型参数。由于早期显存的限制，最早的AlexNet 使用双数据流的设计使一块GPU只需要处理一半模型。
 
**第二，AlexNet 将sigmoid激活函数改成了更加简单的ReLU激活函数。** 
 
 一方面，ReLU激活函数的计算更简单，例如它没有sigmoid激活函数中的求幂运算。
 
 另一方面，ReLU激活函数在不同的参数初始化方法下使模型更容易训练。这是由于当sigmoid激活函数输出极接近0或1时，这些区域的梯度几乎为0，从而造成反向传播无法继续更新部分参数；而ReLU激活函数在正区间的梯度恒为1。因此，若模型参数初始化不当，sigmoid函数可能在正区间内得到几乎为0的梯度，从而令模型无法得到有效的训练。
 
**第三，AlexNet通过丢弃法，来控制全连接层的模型复杂度。而LeNet并没有使用丢弃法**
 
**第四，AlexNet引入了大量的图像增广，如翻转、裁剪和颜色变化，从而进一步扩大数据集来缓解过拟合。**


**AlexNet 与LeNet结构类似，但是使用了更多的卷积层和更大的参数空间来拟合大规模数据集ImageNet。它是浅层神经网络和深度神经网络的分界线。**

AlexNet在LeNet的基础上增加了3个卷积层。但AlexNet作者对它们的卷积窗口、输出通道数和构造顺序做了大量的调整。虽然，AlexNet指明了深度卷积神经网络可以取得出色的结果，但是并没有提供简单的规则以指导后来的研究者如何设计新的网络。


In [1]:
import d2lzh as d2l
from mxnet import gluon,init,nd
from mxnet.gluon import data as gdata,nn
import os
import sys 

net = nn.Sequential()

# 使用较大的 11*11窗口来捕获物体。同时使用步幅4来较大幅度减小输出高和宽。 这里使用输出通道数也比LeNet大很多

net.add(nn.Conv2D(96,kernel_size = 11, strides = 4, activation="relu"),
        nn.MaxPool2D(pool_size = 3, strides = 2),
        
        # 减小卷积窗口，使用填充为2使得输入与输出的高与宽一致，且增大输出通道
        nn.Conv2D(256,kernel_size = 5, padding = 2, activation = 'relu'),
        nn.MaxPool2D(pool_size = 3, strides = 2),
        
        # 前2个卷积层后不使用池化层来减小输入的高和宽
        nn.Conv2D(384, kernel_size = 3, padding = 1, activation = 'relu'),
        nn.Conv2D(384, kernel_size = 3, padding = 1, activation = 'relu'),
        nn.Conv2D(256, kernel_size = 3, padding = 1, activation = 'relu'),
        nn.MaxPool2D(pool_size = 3, strides = 2),
        
        # 这里全连接层的输出个数比LeNet中的大数倍。使用丢弃层来缓解过拟合
        nn.Dense(4096, activation = 'relu'),
        nn.Dropout(0.5),
        nn.Dense(4096, activation = 'relu'), 
        nn.Dropout(0.5),
        
        # 输出层 由于这里使用Fashion_MNIST，所以类别为10
        nn.Dense(10)
       )

In [ ]:
# nn.conv2D:
#     out_height = floor((height+2*padding[0]-dilation[0]*(kernel_size[0]-1)-1)/stride[0])+1
#     out_width = floor((width+2*padding[1]-dilation[1]*(kernel_size[1]-1)-1)/stride[1])+1
    
# nn.MaxPool2D:
#     out_height = floor((height+2*padding[0]-pool_size[0])/strides[0])+1
#     out_width = floor((width+2*padding[1]-pool_size[1])/strides[1])+1


In [89]:
#######################################################卷积层&池化 输出尺寸计算
import numpy as np 

def cal_conv_output_size(n,kernel_size,padding_size,stride):
    if (n + 2*padding_size - (kernel_size - 1))%stride==0:
        return (n + 2*padding_size - (kernel_size - 1))/ stride
    else:
        return np.floor((n + 2*padding_size - (kernel_size - 1))/stride) + 1

def cal_pooling_output_size(n,pool_size,padding_size,stride):
    if (n + 2*padding_size - pool_size) % stride == 0 :
        return (n + 2*padding_size - pool_size) / stride
    else:
        return np.floor((n + 2*padding_size - pool_size) / stride) + 1
    

nh,nw,kh,kw,ph,pw,sh,sw = 224,224,11,11,0,0,4,4
print("conv0 \t Input [nh,hw] = [%d , %d] ; Kernel [kh,kw] = [%d , %d] ; Padding [ph,pw] = [%d , %d] ; Strid: %d "%( nh,nw,kh,kw,ph,pw,sh ))
nh, nw = cal_conv_output_size(nh, kh,ph,sh), cal_conv_output_size(nw,kw,pw,sw)
# nh, nw = np.floor((nh + 2*ph - (kh - 1))/sh) + 1, np.floor((nw + 2*pw - (kw - 1))/sw) + 1
print("\t OutputShape [%d , %d]" %(nh,nw))

# kh,kw = 3,3 # pooling
sh,sw = 2,2 # stride
pool_size = 3

print("Pool0 \t Input [nh,hw] = [%d , %d] ; PoolSize : [%d , %d] ; Padding [ph,pw] = [%d , %d] ; Strid: %d "%( nh,nw,pool_size,pool_size,ph,pw,sh ))
nh, nw = cal_pooling_output_size(nh, pool_size, ph,sh), cal_pooling_output_size(nw, pool_size, pw, sw)
print("\t OutputShape [%d , %d]" %(nh,nw))

kh,kw = 5,5 # kernel 
ph,pw = 2,2
sh,sw = 1,1 # 上面 Conv1中没有定义stride,默认值为1

print("conv1 \t Input [nh,hw] = [%d , %d] ; Kernel [kh,kw] = [%d , %d] ; Padding [ph,pw] = [%d , %d] ; Strid: %d "%( nh,nw,kh,kw,ph,pw,sh ))
nh, nw = cal_conv_output_size(nh, kh,ph,sh), cal_conv_output_size(nw,kw,pw,sw)
print("\t OutputShape [%d , %d]" %(nh,nw))

sh,sw = 2,2
pool_size = 3 
ph,pw = 0,0
print("Pool1 \t Input [nh,hw] = [%d , %d] ; PoolSize : [%d , %d] ; Padding [ph,pw] = [%d , %d] ; Strid: %d "%( nh,nw,pool_size,pool_size,ph,pw,sh ))
nh, nw = cal_pooling_output_size(nh, pool_size, ph,sh), cal_pooling_output_size(nw, pool_size, pw, sw)
print("\t OutputShape [%d , %d]" %(nh,nw))

kh,kw = 3,3
ph,pw = 1,1
sh,sw = 1,1
print("conv2 \t Input [nh,hw] = [%d , %d] ; Kernel [kh,kw] = [%d , %d] ; Padding [ph,pw] = [%d , %d] ; Strid: %d "%( nh,nw,kh,kw,ph,pw,sh ))
nh, nw = cal_conv_output_size(nh, kh,ph,sh), cal_conv_output_size(nw,kw,pw,sw)
print("\t OutputShape [%d , %d]" %(nh,nw))

print("conv3 \t Input [nh,hw] = [%d , %d] ; Kernel [kh,kw] = [%d , %d] ; Padding [ph,pw] = [%d , %d] ; Strid: %d "%( nh,nw,kh,kw,ph,pw,sh ))
nh, nw = cal_conv_output_size(nh, kh,ph,sh), cal_conv_output_size(nw,kw,pw,sw)
print("\t OutputShape [%d , %d]" %(nh,nw))

print("conv4 \t Input [nh,hw] = [%d , %d] ; Kernel [kh,kw] = [%d , %d] ; Padding [ph,pw] = [%d , %d] ; Strid: %d "%( nh,nw,kh,kw,ph,pw,sh ))
nh, nw = cal_conv_output_size(nh, kh,ph,sh), cal_conv_output_size(nw,kw,pw,sw)
print("\t OutputShape [%d , %d]" %(nh,nw))


sh,sw = 2,2
pool_size = 3 
ph,pw = 0,0
print("Pool2 \t Input [nh,hw] = [%d , %d] ; PoolSize : [%d , %d] ; Padding [ph,pw] = [%d , %d] ; Strid: %d "%( nh,nw,pool_size,pool_size,ph,pw,sh ))
nh, nw = cal_pooling_output_size(nh, pool_size, ph,sh), cal_pooling_output_size(nw, pool_size, pw, sw)
print("\t OutputShape [%d , %d]" %(nh,nw))


conv0 	 Input [nh,hw] = [224 , 224] ; Kernel [kh,kw] = [11 , 11] ; Padding [ph,pw] = [0 , 0] ; Strid: 4 
	 OutputShape [54 , 54]
Pool0 	 Input [nh,hw] = [54 , 54] ; PoolSize : [3 , 3] ; Padding [ph,pw] = [0 , 0] ; Strid: 2 
	 OutputShape [26 , 26]
conv1 	 Input [nh,hw] = [26 , 26] ; Kernel [kh,kw] = [5 , 5] ; Padding [ph,pw] = [2 , 2] ; Strid: 1 
	 OutputShape [26 , 26]
Pool1 	 Input [nh,hw] = [26 , 26] ; PoolSize : [3 , 3] ; Padding [ph,pw] = [0 , 0] ; Strid: 2 
	 OutputShape [12 , 12]
conv2 	 Input [nh,hw] = [12 , 12] ; Kernel [kh,kw] = [3 , 3] ; Padding [ph,pw] = [1 , 1] ; Strid: 1 
	 OutputShape [12 , 12]
conv3 	 Input [nh,hw] = [12 , 12] ; Kernel [kh,kw] = [3 , 3] ; Padding [ph,pw] = [1 , 1] ; Strid: 1 
	 OutputShape [12 , 12]
conv4 	 Input [nh,hw] = [12 , 12] ; Kernel [kh,kw] = [3 , 3] ; Padding [ph,pw] = [1 , 1] ; Strid: 1 
	 OutputShape [12 , 12]
Pool2 	 Input [nh,hw] = [12 , 12] ; PoolSize : [3 , 3] ; Padding [ph,pw] = [0 , 0] ; Strid: 2 
	 OutputShape [5 , 5]


In [ ]:
# # test 

# temp_X = nd.random.uniform(shape = (1,1,224,224))
# net.initialize()

# for layer in net:
#     temp_X = layer(temp_X)
#     print(layer.name, "output shape:\t",temp_X.shape)

In [27]:
# 读取数据集

def load_data_fashion_mnist(batch_size, resize = None, root = os.path.join("~",".mxnet",'datasets','fashion-mnist')):

    root = os.path.expanduser(root) # 展开用户路径 ~
    transformer = []
    
    # 读取数据的时候，将图像高和宽扩大到AlexNet中使用的图像大小，即224，这里是通过Resize实现的；
    # 也就是说，在ToTensor实例前使用Resize实例，然后通过Compose实例来将这两个变换串联以方便调用
    
    if resize:
        transformer += [gdata.vision.transforms.Resize(resize)]
        
    transformer += [gdata.vision.transforms.ToTensor()]
    transformer = gdata.vision.transforms.Compose(transformer)
    
    mnist_train = gdata.vision.FashionMNIST(root = root,train = True)
    mnist_test = gdata.vision.FashionMNIST(root = root, train = False)
    
    num_workers = 0 if sys.platform.startswith('win32') else 4 
    train_iter = gdata.DataLoader(mnist_train.transform_first(transformer), batch_size, shuffle = True, num_workers = num_workers)
    test_iter = gdata.DataLoader(mnist_test.transform_first(transformer), batch_size, shuffle = False, num_workers = num_workers)
    
    return train_iter,test_iter
 
batch_size = 128

# 如果出来 "out of memory" 的报错信息，可减小 batch_size 或resize
train_iter, test_iter = load_data_fashion_mnist(batch_size, resize = 224)

In [28]:
# 训练
lr, num_epochs, ctx = 0.01, 10, d2l.try_gpu()
net.initialize(force_reinit = True, ctx = ctx, init = init.Xavier())

trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr})
d2l.train_ch5( net, train_iter, test_iter, batch_size, trainer, ctx, num_epochs)


training on gpu(0)
epoch 1, loss 1.3100, train acc 0.509, test acc 0.749, time 109.8 sec
epoch 2, loss 0.6509, train acc 0.756, test acc 0.801, time 107.0 sec
epoch 3, loss 0.5350, train acc 0.800, test acc 0.833, time 104.3 sec
epoch 4, loss 0.4681, train acc 0.828, test acc 0.856, time 103.5 sec
epoch 5, loss 0.4261, train acc 0.845, test acc 0.866, time 100.6 sec
epoch 6, loss 0.3952, train acc 0.856, test acc 0.874, time 100.4 sec
epoch 7, loss 0.3742, train acc 0.865, test acc 0.879, time 100.4 sec
epoch 8, loss 0.3549, train acc 0.871, test acc 0.883, time 100.5 sec
epoch 9, loss 0.3414, train acc 0.875, test acc 0.889, time 100.9 sec
epoch 10, loss 0.3276, train acc 0.881, test acc 0.892, time 102.8 sec
